In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, make_scorer
from sklearn.feature_selection import SelectFromModel
from sklearn.inspection import permutation_importance
import optuna
from scipy.stats import norm

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch
import numpy as np
import pandas as pd

In [5]:


class ImprovedAlertPredictor:
    def __init__(self, prediction_horizon=30, confidence_threshold=0.8):
        self.prediction_horizon = prediction_horizon
        self.confidence_threshold = confidence_threshold
        self.model = None
        self.scaler = None
        self.feature_selector = None
        self.important_features = None

    def custom_scorer(self, y_true, y_pred):
        """
        Custom scoring function that penalizes false positives more heavily
        """
        base_mae = mean_absolute_error(y_true, y_pred)
        # Penalize predictions that are too early (false positives)
        early_predictions = y_pred < y_true
        false_positive_penalty = np.mean(np.abs(y_true[early_predictions] - y_pred[early_predictions])) * 1.5
        return -(base_mae + false_positive_penalty)

    def engineer_additional_features(self, df):
        """
        Add more sophisticated features to improve model performance
        """
        # Exponential moving averages with different windows
        df['ema_12'] = df.groupby('machine_id')['ChlPrs'].ewm(span=12).mean()
        df['ema_24'] = df.groupby('machine_id')['ChlPrs'].ewm(span=24).mean()

        # Pressure change rates
        df['pressure_change_rate'] = df.groupby('machine_id')['ChlPrs'].diff()
        df['pressure_change_rate_ma'] = df.groupby('machine_id')['pressure_change_rate'].rolling(12).mean()

        # Time-based cyclical features
        df['hour_sin'] = np.sin(2 * np.pi * df['hour']/24)
        df['hour_cos'] = np.cos(2 * np.pi * df['hour']/24)
        df['day_sin'] = np.sin(2 * np.pi * df['day_of_week']/7)
        df['day_cos'] = np.cos(2 * np.pi * df['day_of_week']/7)

        # Volatility features
        df['volatility'] = df.groupby('machine_id')['ChlPrs'].rolling(24).std()

        return df

    def optimize_hyperparameters(self, X_train, y_train):
        """
        Use Optuna for hyperparameter optimization
        """
        def objective(trial):
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 500),
                'max_depth': trial.suggest_int('max_depth', 5, 30),
                'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
                'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
                'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
            }

            model = RandomForestRegressor(**params, random_state=42)
            scores = cross_val_score(model, X_train, y_train,
                                   scoring=make_scorer(self.custom_scorer),
                                   cv=5)
            return scores.mean()

        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=50)
        return study.best_params

    def select_features(self, X, y):
        """
        Perform feature selection using permutation importance
        """
        base_model = RandomForestRegressor(n_estimators=100, random_state=42)
        base_model.fit(X, y)

        perm_importance = permutation_importance(base_model, X, y, n_repeats=10, random_state=42)
        important_features = X.columns[perm_importance.importances_mean > np.mean(perm_importance.importances_mean)]

        return important_features

    def fit(self, X, y):
        """
        Train the model with all improvements
        """
        # Scale features
        self.scaler = StandardScaler()
        X_scaled = self.scaler.fit_transform(X)
        X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

        # Select important features
        self.important_features = self.select_features(X_scaled, y)
        X_selected = X_scaled[self.important_features]

        # Optimize hyperparameters
        best_params = self.optimize_hyperparameters(X_selected, y)

        # Train final model
        self.model = RandomForestRegressor(**best_params, random_state=42)
        self.model.fit(X_selected, y)

        return self

    def predict(self, X, return_confidence=False):
        """
        Make predictions with confidence estimates
        """
        X_scaled = self.scaler.transform(X)
        X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
        X_selected = X_scaled[self.important_features]

        # Get predictions from all trees
        predictions = np.array([tree.predict(X_selected)
                              for tree in self.model.estimators_])

        # Calculate mean and confidence interval
        mean_pred = np.mean(predictions, axis=0)
        std_pred = np.std(predictions, axis=0)
        confidence = 1 - 2 * norm.cdf(-np.abs(mean_pred - np.mean(predictions, axis=0)) / std_pred)

        # Filter predictions based on confidence threshold
        mean_pred[confidence < self.confidence_threshold] = self.prediction_horizon

        if return_confidence:
            return mean_pred, confidence
        return mean_pred


# Prepare data for model training
def prepare_data_for_model(df, target_alert_type, prediction_horizon):
    df['target'] = df.groupby('machine_id').apply(lambda x: x[x['ALERT'] == target_alert_type]['Time'].shift(-1) - x['Time']).reset_index(level=0, drop=True)
    df['target'] = df['target'].dt.total_seconds() / (24 * 3600)  # Convert to days

    # Remove rows where target is greater than the prediction horizon
    df = df[df['target'] <= prediction_horizon]

    # Remove rows with NaN targets (i.e., last occurrence of each alert type for each machine)
    df = df.dropna(subset=['target'])

    features = ['ChlPrs', 'hour', 'day_of_week', 'month', 'is_weekend', 'rolling_mean', 'rolling_std'] + [f'time_since_{at}' for at in ['LOW', 'MEDIUM', 'HIGH', 'SIGMA']]
    X = df[features]
    y = df['target']

    return X, y

def train_improved_model(df, target_alert_type, prediction_horizon=30):
    """
    Main function to train the improved model
    """
    # Engineer additional features
    df = ImprovedAlertPredictor().engineer_additional_features(df)

    # Prepare data
    X, y = prepare_data_for_model(df, target_alert_type, prediction_horizon)

    # Initialize and train improved model
    model = ImprovedAlertPredictor(prediction_horizon=prediction_horizon)
    model.fit(X, y)

    return model

# Example usage
def evaluate_improved_model(df, target_alert_type):
    # Split data
    X, y = prepare_data_for_model(df, target_alert_type, prediction_horizon=30)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train improved model
    model = ImprovedAlertPredictor()
    model.fit(X_train, y_train)

    # Make predictions with confidence
    y_pred, confidence = model.predict(X_test, return_confidence=True)

    # Calculate metrics
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    # Calculate false positive rate
    false_positives = np.sum((y_pred < y_test) & (confidence >= model.confidence_threshold))
    total_predictions = len(y_pred)
    false_positive_rate = false_positives / total_predictions

    return {
        'mae': mae,
        'rmse': rmse,
        'false_positive_rate': false_positive_rate,
        'model': model
    }

In [6]:


class AlertVisualization:
    def __init__(self, figure_size=(20, 15)):
        self.figure_size = figure_size
        self.colors = {
            'actual': '#1f77b4',      # Blue
            'predicted': '#ff7f0e',    # Orange
            'confidence': '#2ecc71',   # Green
            'false_positive': '#e74c3c' # Red
        }

    def create_visualization_grid(self, df, improved_model, target_alert_type,
                                prediction_horizon):
        """
        Creates a comprehensive grid of visualizations showing model performance
        """
        # Create figure with subplots
        fig = plt.figure(figsize=self.figure_size)
        gs = plt.GridSpec(3, 2, height_ratios=[2, 1, 1])

        # Main timeline plot
        ax_main = fig.add_subplot(gs[0, :])
        self.plot_alerts_timeline(ax_main, df, improved_model, target_alert_type,
                                prediction_horizon)

        # Confidence distribution plot
        ax_conf = fig.add_subplot(gs[1, 0])
        self.plot_confidence_distribution(ax_conf, df, improved_model,
                                       target_alert_type)

        # Prediction error distribution
        ax_error = fig.add_subplot(gs[1, 1])
        self.plot_prediction_error_distribution(ax_error, df, improved_model,
                                             target_alert_type)

        # Performance metrics
        ax_metrics = fig.add_subplot(gs[2, :])
        self.plot_performance_metrics(ax_metrics, df, improved_model,
                                   target_alert_type)

        plt.tight_layout()
        return fig

    def plot_alerts_timeline(self, ax, df, improved_model, target_alert_type,
                           prediction_horizon):
        """
        Enhanced timeline plot showing actual vs predicted alerts with confidence
        """
        machines = df['machine_id'].unique()

        # Prepare predictions and confidence scores
        features = improved_model.important_features
        X = df[features]
        predictions, confidence = improved_model.predict(X, return_confidence=True)
        df['predicted_days_to_alert'] = predictions
        df['prediction_confidence'] = confidence
        df['predicted_alert_date'] = df['Time'] + pd.to_timedelta(
            df['predicted_days_to_alert'], unit='D')

        # Plot for each machine
        for i, machine_id in enumerate(machines):
            machine_df = df[df['machine_id'] == machine_id]

            # Plot actual alerts
            actual = machine_df[machine_df['ALERT'] == target_alert_type]
            ax.scatter(actual['Time'], [i-0.2] * len(actual),
                      marker='o', s=100, c=self.colors['actual'],
                      label='Actual Alert' if i == 0 else "")

            # Plot predicted alerts with confidence
            predicted = machine_df[
                machine_df['predicted_days_to_alert'] <= prediction_horizon]

            # Color predictions based on confidence
            high_conf = predicted[
                predicted['prediction_confidence'] >=
                improved_model.confidence_threshold]
            low_conf = predicted[
                predicted['prediction_confidence'] <
                improved_model.confidence_threshold]

            # Plot high confidence predictions
            if len(high_conf) > 0:
                ax.scatter(high_conf['predicted_alert_date'],
                          [i+0.2] * len(high_conf),
                          marker='x', s=100, c=self.colors['predicted'],
                          label='High Confidence Prediction' if i == 0 else "")

            # Plot low confidence predictions
            if len(low_conf) > 0:
                ax.scatter(low_conf['predicted_alert_date'],
                          [i+0.2] * len(low_conf),
                          marker='x', s=100, c=self.colors['false_positive'],
                          alpha=0.5,
                          label='Low Confidence Prediction' if i == 0 else "")

            # Add machine labels
            ax.text(df['Time'].min(), i, machine_id, va='center', ha='right',
                   fontweight='bold')

        # Customize plot
        ax.set_yticks(range(len(machines)))
        ax.set_yticklabels([""] * len(machines))
        ax.set_xlabel('Date')
        ax.set_ylabel('Machine ID')
        ax.set_title(
            f'Actual vs Predicted {target_alert_type} Alerts with Confidence')
        ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        ax.grid(True, alpha=0.3)

    def plot_confidence_distribution(self, ax, df, improved_model,
                                  target_alert_type):
        """
        Plot distribution of prediction confidence scores
        """
        features = improved_model.important_features
        X = df[features]
        _, confidence = improved_model.predict(X, return_confidence=True)

        sns.histplot(confidence, ax=ax, bins=30, color=self.colors['confidence'])
        ax.axvline(improved_model.confidence_threshold, color='r', linestyle='--',
                  label=f'Threshold ({improved_model.confidence_threshold})')

        ax.set_title('Distribution of Prediction Confidence Scores')
        ax.set_xlabel('Confidence Score')
        ax.set_ylabel('Count')
        ax.legend()

    def plot_prediction_error_distribution(self, ax, df, improved_model,
                                        target_alert_type):
        """
        Plot distribution of prediction errors
        """
        features = improved_model.important_features
        X = df[features]
        y_true = df['target']
        y_pred = improved_model.predict(X)
        errors = y_pred - y_true

        sns.histplot(errors, ax=ax, bins=30, color=self.colors['predicted'])
        ax.axvline(0, color='k', linestyle='--', label='No Error')

        ax.set_title('Distribution of Prediction Errors')
        ax.set_xlabel('Prediction Error (days)')
        ax.set_ylabel('Count')
        ax.legend()

    def plot_performance_metrics(self, ax, df, improved_model, target_alert_type):
        """
        Plot key performance metrics
        """
        # Calculate metrics
        features = improved_model.important_features
        X = df[features]
        y_true = df['target']
        y_pred = improved_model.predict(X)

        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))

        # Calculate false positive rate
        false_positives = np.sum(y_pred < y_true)
        total_predictions = len(y_pred)
        false_positive_rate = false_positives / total_predictions

        # Plot metrics as a horizontal bar chart
        metrics = {
            'MAE (days)': mae,
            'RMSE (days)': rmse,
            'False Positive Rate': false_positive_rate
        }

        colors = [self.colors['predicted'], self.colors['predicted'],
                 self.colors['false_positive']]

        y_pos = np.arange(len(metrics))
        ax.barh(y_pos, list(metrics.values()), color=colors)
        ax.set_yticks(y_pos)
        ax.set_yticklabels(list(metrics.keys()))

        # Add value labels
        for i, v in enumerate(metrics.values()):
            ax.text(v, i, f'{v:.3f}', va='center')

        ax.set_title('Model Performance Metrics')
        ax.grid(True, alpha=0.3)

# Function to create and display all visualizations
def visualize_improved_results(df, improved_model, target_alert_type,
                             prediction_horizon=30):
    """
    Create and display comprehensive visualization of model results
    """
    visualizer = AlertVisualization()
    fig = visualizer.create_visualization_grid(
        df, improved_model, target_alert_type, prediction_horizon)
    return fig

# Example usage
if __name__ == "__main__":
    # Assuming df and improved_model are already created
    fig = visualize_improved_results(df, improved_model, 'HIGH')
    plt.show()

NameError: name 'df' is not defined